In [166]:
import pandas as pd
import requests

In [57]:
#Importer le json avec un appel vers l'API
response = requests.get('https://data.airbreizh.asso.fr/geoserver/ind_bretagne_agglo/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=ind_bretagne_agglo:ind_bretagne_agglo&outputFormat=application%2Fjson').json()

In [114]:
#Créer un dictionnaire pour renommer les villes
villes = {'CA Saint-Brieuc Armor': 'Saint-Brieuc', 'CA Golfe du Morbihan - Vannes': 'Vannes',
        'CA Quimper Bretagne Occidentale': 'Quimper', 'Rennes Métropole': 'Rennes',
        'CA du Pays de Saint-Malo (Saint-Malo Agglomération)': 'Saint-Malo',
        'Brest Métropole': 'Brest', 'CA Lorient': 'Lorient', }

In [181]:
#Transformer le json en un tableau pandas
data = pd.json_normalize(response, record_path='features')
#Garder seulement les colonnes qui nous intéresse
data = data[['geometry.coordinates', 'properties.date_ech', 'properties.valeur', 'properties.lib_zone']]
#Passer la date au format YYYY-MM-DD
data['properties.date_ech'] = data['properties.date_ech'].apply(lambda x: x[:10])

In [128]:
data['properties.lib_zone'].unique()

array(['CA Saint-Brieuc Armor', 'CA Golfe du Morbihan - Vannes',
       'CA Quimper Bretagne Occidentale', 'Rennes Métropole',
       'CA du Pays de Saint-Malo (Saint-Malo Agglomération)',
       'Brest Métropole', 'CA Lorient'], dtype=object)

In [179]:
from air import Connexion
from pyproj import Proj, transform

In [180]:
json = ''
#Créer des objets Proj pour transformer les coordonnées géographiques
inProj = Proj(init='epsg:2154')
outProj = Proj(init='epsg:4326')

#Créer un json propre et le stocker dans une string
for i, feature in enumerate(response['features']):
    long, lat = feature['geometry']['coordinates']
    long, lat = transform(inProj,outProj,long, lat)
    coordinates = [long, lat]
    
    data = {'date': feature['properties']['date_ech'][:10],
            'ville': villes[feature['properties']['lib_zone']], 
            'niveau': feature['properties']['valeur'],
            'coordinates': coordinates}
    data = str(data).replace("\'", "\"")
    json = json + data

In [178]:
#Importer la string dans un fichier json
with open('json.json', 'w') as f:
    f.write(json)